In [ ]:
!pip install transformers torch

In [ ]:
# 👇 Paste your transcript in this variable (keep the quotes)
conversation = """
Doctor: How are you feeling today?
Patient: I had a car accident. My neck and back hurt a lot for four weeks.
Doctor: Did you receive treatment?
Patient: Yes, I had ten physiotherapy sessions, and now I only have occasional back pain.
"""

In [ ]:
patient_lines = []
for line in conversation.split("\n"):
    if line.lower().startswith("patient:"):
        # Remove "Patient:" and extra spaces
        patient_lines.append(line.split(":", 1)[1].strip())

print("🩺 Extracted Patient Lines:")
for p in patient_lines:
    print("-", p)


In [ ]:
from transformers import pipeline

# Sentiment analysis (general emotional tone)
sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Zero-shot intent classification (no dataset required)
intent_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Define possible intents
candidate_intents = [
    "Seeking reassurance",
    "Reporting symptoms",
    "Expressing concern",
    "Reporting progress"
]

In [ ]:
import json

results = []

for text in patient_lines:
    # Sentiment
    sentiment_raw = sentiment_analyzer(text)[0]['label']
    sentiment = "Anxious" if sentiment_raw == "NEGATIVE" else "Reassured"

    # Intent
    intent_result = intent_classifier(text, candidate_labels=candidate_intents)
    intent = intent_result['labels'][0]

    results.append({
        "Patient_Text": text,
        "Sentiment": sentiment,
        "Intent": intent
    })

print(json.dumps(results, indent=2))

In [ ]:
with open("analysis_results.json", "w") as f:
    json.dump(results, f, indent=2)

print("✅ Results saved to analysis_results.json")